In [ ]:
#entre tour 24 23 
from hashlib import sha256
from Crypto.Cipher import AES

def derive_tree(seed_hex):
    A = [b''] * 2048
    A[1] = bytes.fromhex(seed_hex)

    # construire l’arbre binaire (niveau 1 → 1023)
    for i in range(1, 1024):
        h = sha256(A[i]).digest()
        A[2*i]   = h[:16]
        A[2*i+1] = h[16:]
    return A

def make_commitment(seed_hex):
    A = derive_tree(seed_hex)

    # les 1024 clefs K[i] sont les feuilles A[1024..2047]
    buf = b''
    for i in range(1024):
        K = A[i+1024]
        cipher = AES.new(K, AES.MODE_ECB)
        X = cipher.encrypt(b'\x00'*16)
        Y = cipher.encrypt(b'\xff'*16)
        buf += X + Y

    commit = sha256(buf).hexdigest()
    return commit


seed = "416e746f696e654d616c6c65726574"
commitment = make_commitment(seed)
print("Commitment =", commitment)


Commitment = 49589336e9caf0cb685a37ef9d7895a44256081c8e2b235915c0b5825df9b384


In [ ]:
from hashlib import sha256
from Crypto.Cipher import AES

def derive_tree(seed_hex):
    A = [b''] * 2048
    A[1] = bytes.fromhex(seed_hex)
    for i in range(1, 1024):
        h = sha256(A[i]).digest()
        A[2*i]   = h[:16]
        A[2*i+1] = h[16:]
    return A

def get_pair(seed_hex, i):
    A = derive_tree(seed_hex)
    K = A[1024 + i]
    cipher = AES.new(K, AES.MODE_ECB)
    X = cipher.encrypt(b'\x00' * 16)
    Y = cipher.encrypt(b'\xff' * 16)
    return X.hex(), Y.hex()

⇩
seed = "416e746f696e654d616c6c65726574"   

i = 42

X, Y = get_pair(seed, i)
print(f"X[{i}] =", X)
print(f"Y[{i}] =", Y)


X[42] = b0b7c238349cbeafde52b0877d4dfebb
Y[42] = 8e7089dcfa09b42069026d473e446084


In [6]:
from hashlib import sha256
from Crypto.Cipher import AES

def derive_tree(seed_hex):
    A = [b''] * 2048
    A[1] = bytes.fromhex(seed_hex)
    for i in range(1, 1024):
        h = sha256(A[i]).digest()
        A[2*i]   = h[:16]
        A[2*i+1] = h[16:]
    return A

def copath(seed_hex, i):
    A = derive_tree(seed_hex)
    path = []
    j = i + 1024
    for _ in range(10):
        path.append(A[j ^ 1].hex())
        j //= 2
    return path

seed = "416e746f696e654d616c6c65726574"
i = 42
for k, val in enumerate(copath(seed, i)):
    print(f"[{k:3}] : {val}")


[  0] : 6ebdd1aac3df4868dcd14f5d0fdeb960
[  1] : 46f17d541162ecd5983b77c8c39f7905
[  2] : b35d926d66084cffe7b5e9b748528cc4
[  3] : 6b5caf4ae4de8fc8b45ce1cf9030332f
[  4] : 4760121edd2dffc8b0da9e5492ea4678
[  5] : 6537c09aa91eb69f7931084525218a12
[  6] : 46ff948547843d096803be653400713e
[  7] : f9e67d47a3f0b335cc657fad96695cd1
[  8] : 7bf649170aa274404f1351734d79879a
[  9] : 7b498ed39011db1b17bef5cb18e492a1


Signature : 303502187B20238C90354BEF32638D22D315B53B3589136EA48A808F021900BD102D57ADD695CBB86BFB8FE0013D61485593C337445BC2

In [ ]:
#Borne d'arcade, PFC 1
import secrets

# paramètres du groupe Oakley 14
g = 2
p_hex = """FFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E08
8A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD
3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E
7EC6F44C42E9A63A36210000000000090563"""  
p = int(p_hex.replace("\n", ""), 16)

# secret privé
x = secrets.randbelow(p-2) + 1
h = pow(g, x, p)

print("Ta commande à envoyer :")
print("PKEY", format(h, "x"))


Ta commande à envoyer :
PKEY d8f4f2d038e70cf0c73369cd1d3fd9367cd2ef705b369e43e5f0d1d0d8394623115f03c0214bc830e36cc8d86c0aa0ad07292f30e258bff226cfb8faa88d7fe3bed333fb63ee777ec68f1a82d53a7501a9c25cc17bcf4be62f2770ecad2330a


In [ ]:
import secrets

# Paramètres du groupe Oakley 14 (2048 bits)
g = 2
p_hex = """FFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E08
8A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD
3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E
7EC6F44C42E9A63A36210000000000090563"""
p = int(p_hex.replace("\n",""),16)

# Choisir le coup : "PIERRE", "FEUILLE", ou "CISEAUX"
coup = "PIERRE"

# Mapper le coup sur un entier (ex: 0, 1, 2)
#encodage de PIERRE en hexa à la place de 0 
mapping = {"PIERRE": 0, "FEUILLE": 1, "CISEAUX": 2}
m = mapping[coup]

# Générer la clé privée pour le commit (aléa k)
k = secrets.randbelow(p-2) + 1

# COMMIT ElGamal : r = g^k mod p, c = m * h^k mod p
# Ici h est ta clé publique (déclarée dans PKEY)
# Pour exemple on génère une clé publique temporaire h
x = secrets.randbelow(p-2) + 1
h = pow(g, x, p)

r = pow(g, k, p)
c = (m * pow(h, k, p)) % p

# Afficher les commandes à envoyer au jeu
print("COMMIT", format(r, "x"), format(c, "x"))
print("MOVE", coup)
print("OPEN", format(k, "x"))


COMMIT cd68c00ac0b2eac81836bfe2047559ba0de9f22c66b8efde0780ba9ccb94bdc8bb3fda38df4dd926291e30d6cbefb21f8144c166759c117cfecb9077787ae58385cc3821c482d9d14152dfe6051066305e094fefa8b3511eb9ef3302a8b8d336 0
MOVE PIERRE
OPEN 88fc97ad4fe56d2c5427b44a5386fa519b4f153d23e0cf5f439a0b3aaef6378e5d8cd0a9f71daac42f3c0bf244544233c2a2199ed55e020f046c7f9b32b9a97f2cec64cbf5300bcb0729a6ad588f714f729bf38639d760869853e1e67d48fccf
